<a href="https://colab.research.google.com/github/msakarvadia/Machine-Learning/blob/master/LC_LoanDefaultRatePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
#!pwd
%cd drive/My\ Drive
%cd Colab\ Notebooks
%cd Loan\ Project
# we are in the "Loan Project directory" - if there is an issue use "%cd ... to reenter directory"
#!ls
#!cat LoanStats3a_securev1_LendingClub_07-11.csv

[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive/Colab Notebooks/Loan Project
[Errno 2] No such file or directory: 'Colab Notebooks'
/content/drive/My Drive/Colab Notebooks/Loan Project
[Errno 2] No such file or directory: 'Loan Project'
/content/drive/My Drive/Colab Notebooks/Loan Project


In [59]:
#data is from Lending Club - I have renamed csv files
df_07_11 = pd.read_csv('LoanStats3a_securev1_LendingClub_07-11.csv', sep=",", skiprows=1)
df_12_13 = pd.read_csv('LoanStats3b_securev1_LendingClub_12-13.csv', sep=",", skiprows=1)
df_14 = pd.read_csv("LoanStats3c_LendingClub_2014.csv", sep=",", skiprows=1)
df_15 = pd.read_csv('LoanStats3d_LendingClub_2015.csv', sep=",", skiprows=1)
df_16Q1 = pd.read_csv('LoanStats_LendingClub_2016Q1.csv', sep=",", skiprows=1)
df_16Q2 = pd.read_csv('LoanStats_LendingClub_2016Q2.csv', sep=",", skiprows=1)
df_16Q3 = pd.read_csv('LoanStats_LendingClub_2016Q3.csv', sep=",", skiprows=1)
df_16Q4 = pd.read_csv('LoanStats_LendingClub_2016Q4.csv', sep=",", skiprows=1)
df_17Q1 = pd.read_csv('LoanStats_LendingClub_2017Q1.csv', sep=",", skiprows=1)
df_17Q2 = pd.read_csv('LoanStats_LendingClub_2017Q2.csv', sep=",", skiprows=1)
df_17Q3 = pd.read_csv('LoanStats_LendingClub_2017Q3.csv', sep=",", skiprows=1)
df_17Q4 = pd.read_csv('LoanStats_LendingClub_2017Q4.csv', sep=",", skiprows=1)
df_18Q1 = pd.read_csv('LoanStats_LendingClub_2018Q1.csv', sep=",", skiprows=1)
df_18Q2 = pd.read_csv('LoanStats_LendingClub_2018Q2.csv', sep=",", skiprows=1)
df_18Q3 = pd.read_csv('LoanStats_LendingClub_2018Q3.csv', sep=",", skiprows=1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, 

In [0]:
frames = [df_07_11, df_12_13, df_14, df_15, df_16Q1, df_16Q2, df_16Q3, df_16Q4, df_17Q1, df_17Q2, df_17Q3, df_17Q4, df_18Q1,df_18Q2,df_18Q3]

result = pd.concat(frames, ignore_index=True)

In [61]:
result.shape

(2132287, 151)

In [62]:
result.drop(columns=['id', 'member_id','url','zip_code'], inplace=True)
result.shape

(2132287, 147)

In [63]:
#"loan_status" is the column heading of whether it defulted or not
x = result.copy()
#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()
y.shape[0]

2132287

In [64]:
label = []
for i in range(y.shape[0]):
  if y[i] not in label:
    label.append(y[i])

for i in range(len(label)):
  print(label[i])
  
print("There are", len(label), "labels")

Fully Paid
Charged Off
nan
Does not meet the credit policy. Status:Fully Paid
Does not meet the credit policy. Status:Charged Off
Late (31-120 days)
Current
Late (16-30 days)
In Grace Period
Default
There are 10 labels


##NOTES about loan_status (aka. labels):


*   if label is "nan" - we throw that loan out of the training set
*   Current means that the loan is still being paid off, and therefore we cannot use for out training - might be useful to comeback and revisit in the future (maybe predict outcome now) - will be thrown out of training set
*   fully paid  - successful loan


---


*   what is "charged off" vs "default"
 *   "Loans that are in "Default" are loans for which borrowers have failed to make payments for an extended period of time. A loan becomes “Charged Off” when there is no longer a reasonable expectation of further payments.  Charge Off typically occurs when a loan is 120 days or more past due and there is no reasonable expectation of sufficient payment to prevent the charge off.  In certain circumstances, loans may be charged off at an earlier or later date.  Please note, loans for which borrowers have filed for bankruptcy may be charged off earlier based on the date of bankruptcy notification.
 A loan that is in “Default” will still appear in your Notes, in the status of “Default,” while a loan that has been “Charged Off” will appear as charged off, and the remaining principal balance of the Note will be deducted from your account balance.  Learn more about what happens when a loan is charged off"
 *   Because in the case of "charged off" vs "default" the lender has extremely low chance of recovering their investment, we will treat both of these classifications as if they are the same thing

---


*   what does it mean if it "does not meet the credit policy"
 *   Lending Club doesn't make clear the meaning of this statement, but from general internet research most ppl just ommit the "does not meet the credit policy" part and use the second half of the classification
---
*   what does "grace period" mean - should we potentially throw those away?
 * Loan is past due but within the 15-day grace period. - we will throw these away because for training data it isn't useful since the final status of loan isn't avaliable
*   if its late - does that mean status is still being finalized? - should we throw away?
 * if it is late we don't use this loan in training set cus finalized status isn't avalible to confirm predictions



In [65]:
result = result[~result['loan_status'].isin(['Late (31-120 days)', 'Current', 'Late (16-30 days)', 'In Grace Period'])]
# above gets rid of rows that have loan status' that we don't want to consider
result = result[~result['loan_status'].isna()] 

#make labels binary
pay_status = {'Fully Paid': 1, 'Charged Off': 0, 'Does not meet the credit policy. Status:Fully Paid':1, 'Does not meet the credit policy. Status:Charged Off':0, 'Default':0}
result.loan_status = [pay_status[item] for item in result.loan_status]

# above gets rid of rows that don't have a valid loan status
result.shape

(1201050, 147)

In [66]:
#"loan_status" is the column heading of whether it defulted or not
x = result.copy()
#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()
#get rid of labels from test set
x.drop(columns=['loan_status'], inplace=True) 

#for now, we are gonna drop all columns that have even one NAN value - this signifigantly reduces training data (so its something we want to comeback and deal with later)
x.dropna(axis='columns', inplace=True)
print(list(x))

#y #can print out labels
x.shape
y

['addr_state', 'application_type', 'collection_recovery_fee', 'debt_settlement_flag', 'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'purpose', 'pymnt_plan', 'recoveries', 'revol_bal', 'sub_grade', 'term', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'verification_status']


0          1
1          0
2          1
3          1
4          1
5          1
6          1
7          1
8          0
9          0
10         1
11         1
12         0
13         1
14         0
15         1
16         1
17         1
18         1
19         1
20         1
21         0
22         1
23         1
24         0
25         1
26         0
27         0
28         1
29         1
          ..
2131612    1
2131662    1
2131668    1
2131674    1
2131677    1
2131678    1
2131686    1
2131687    1
2131738    1
2131818    1
2131825    1
2131828    1
2131854    1
2131883    1
2131898    1
2131911    1
2131952    1
2132045    1
2132053    1
2132071    1
2132079    1
2132108    1
2132113    1
2132131    1
2132132    1
2132180    1
2132186    1
2132188    1
2132213    1
2132273    1
Name: loan_status, Length: 1201050, dtype: int64

In [0]:
#y = np_utils.to_categorical(y)

In [68]:
print(y[13])
y.shape

1


(1201050,)

In [0]:
x['term'] = x['term'].map(lambda x: x.lstrip('%').rstrip('months'))
x['int_rate'] = x["int_rate"].map(lambda x: x.rstrip("%"))
#x['revol_util'] = x['revol_util'].str.replace('%', '') #format changes bc column as floats and ints, also has NaN values, so it was removed above
#print(x["revol_util"])

In [70]:
#dropping non numerical columns for non - will/should come back a one hot encode categorical columns
x = x.select_dtypes(['number'])
x

,collection_recovery_fee,funded_amnt,funded_amnt_inv,installment,last_pymnt_amnt,loan_amnt,out_prncp,out_prncp_inv,policy_code,recoveries,revol_bal,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp
0,0.0000,5000.0,4975.000000,162.87,171.62,5000.0,0.0,0.0,1.0,0.00,13648.0,5863.155187,5833.84,863.16,0.00,5000.00
1,1.1100,2500.0,2500.000000,59.83,119.66,2500.0,0.0,0.0,1.0,122.90,1687.0,1014.530000,1014.53,435.17,0.00,456.46
2,0.0000,2400.0,2400.000000,84.33,649.91,2400.0,0.0,0.0,1.0,0.00,2956.0,3005.666844,3005.67,605.67,0.00,2400.00
3,0.0000,10000.0,10000.000000,339.31,357.48,10000.0,0.0,0.0,1.0,0.00,5598.0,12231.890000,12231.89,2214.92,16.97,10000.00
4,0.0000,3000.0,3000.000000,67.79,67.30,3000.0,0.0,0.0,1.0,0.00,27783.0,4066.908161,4066.91,1066.91,0.00,3000.00
5,0.0000,5000.0,5000.000000,156.46,161.03,5000.0,0.0,0.0,1.0,0.00,7963.0,5632.210000,5632.21,632.21,0.00,5000.00
6,0.0000,7000.0,7000.000000,170.08,1313.76,7000.0,0.0,0.0,1.0,0.00,17726.0,10137.840008,10137.84,3137.84,0.00,7000.00
7,0.0000,3000.0,3000.000000,109.43,111.34,3000.0,0.0,0.0,1.0,0.00,8221.0,3939.135294,3939.14,939.14,0.00,3000.00
8,2.0900,5600.0,5600.000000,152.39,152.39,5600.0,0.0,0.0,1.0,190.54,5210.0,647.500000,647.50,294.94,0.00,162.02
9,2.5200,5375.0,5350.000000,121.45,121.45,5375.0,0.0,0.0,1.0,277.69,9279.0,1484.590000,1477.70,533.42,0.00,673.48


In [71]:
print(list(x))
x.shape

['collection_recovery_fee', 'funded_amnt', 'funded_amnt_inv', 'installment', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'recoveries', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp']


(1201050, 16)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

seed = 45
np.random.seed(seed)

In [0]:

my_model = Sequential()
my_model.add(Dense(150, input_dim=16, activation='relu'))
my_model.add(Dense(150,  activation='relu'))
my_model.add(Dense(150,  activation='relu'))
my_model.add(Dense(150,  activation='relu'))
my_model.add(Dense(1, activation='softmax')) #when i one hot encode the answer key - u need output nueron number to be 2 not 1

my_model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) #need to one hot encode the answer key cus im a dumb dumb

In [80]:
scores = my_model.evaluate(X_train, y_train, verbose=0)
print("Accuracy on training data: %.2f%%" % (scores[1]*100))

scores = my_model.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

Accuracy on training data: 79.92%
Accuracy on test data: 79.95%


In [0]:
y_pred = my_model.predict(X_train)

In [85]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred)

#model is never predicting that the person will not pay

array([[     0, 161562],
       [     0, 643141]])